**TO DO: 
    1) EDIT INTRO TEXT
    2) UPDATE ALL CODE SO THAT EXAMPLES CAN BE INTERCHANGED AND ADD INSTRUCTIONS TO CHANGE UP MODELS
    3) MAKE SURE FIGURES ARE THE SAME AS THOSE IN THE PAPER
    4) ADD INSTRUCTIONS/EXPLANATIONS THROUGHOUT**
    
    NOTE: THIS WAS MADE USING THE "Case 4 - Test in large watershed - intermittent rainfall - No bed evolution" DRIVER FILE

<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

# Test comparing the four different models in riverBedDynamics
This notebook was created by Sam Anderson, Mikey Sison and Angel Monsalve.

<hr>
<small>For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

**What is this notebook?**

All our previous bed evolution tests had a predominant flow direction and were restricted to pure erosion or deposition. We conducted a final test of our LEM in a more complex and larger watershed to analyze the how flow discharge and bed surface elevation varies at different locations within the domain under different rainfall events. We used the synthetic square watershed from Adams et al. (2017) which has an area of 36 km2 with a resolution of 30 x 30 m per pixel and elevations ranging from 0 m at the basin outlet to 225 m at the highest point (Figure 10 a). Two cases of temporal distribution of rainfall intensity were considered, both having the same total volume of water precipitated (10 mm). We referred to these cases as i) uniform, in which the rainfall intensity was 5 mm/hr lasting for two hours and ii) intermittent, where rainfall can reach up to 25 mm/hr during 350 s and then goes back to zero (Figure 10 b). Changes in flow discharge and bed surface elevation were quantified in three locations: Site 1 which coincides with the watershed outlet, Site 2 located upstream the outlet and upstream the confluence of the most downstream tributaries, and Site 3 located approximately at the center of the watershed (Figure 10 a). 

We ran each model for 24 hours, set Manning’s n constant in the whole watershed with a value of 0.035, set the option for  steep_slopes to True in OverlandFlow, used the bed load transport equation of Meyer-Peter & Müller (1948) with a D_50 of 32 mm, and allowed the critical shear stress to vary spatially using the equation of  Mueller et al. (2005) (variableCriticalShearStress = True). All other variables during the instantiation of the components had default values. Each rainfall intensity case was simulated with and without activating riverBedDynamics (4 cases in total) to analyze the effect that the selected temporal distribution of rainfall intensity has on flow hydraulics (e.g., flow discharge) and in turn on morphological changes. When running only OverlandFlow (i.e., riverBedDynamics deactivated) the resulting hydrograph for both the uniform and intermittent cases have a relatively smooth shape at the three sites (Figure 10 b). In the intermittent case, the peak discharge arrives first at every site and has a larger magnitude (50.9 m3/s arriving after 2.1 hours compared to 42.8 m3/s at 2.6 hours for the uniform case). If the bed surface elevation evolves as a function of the local flow conditions (i.e., riverBedDynamics is activated) the resulting hydrograph had a lower peak discharge. At the outlet and at Site 2 the reductions are nearly 15% and 33% for the uniform and intermittent case, respectively (Figure 10 b). At Site 3 the changes in hydrograph shape are relatively small with discharge peak decreasing from 14.7 to 14.2 m3/s in the uniform case and from 20.3 to 18.9 m3/s in the intermittent scenario. Additionally, the shape of the hydrograph at sites 1 and 2 is no longer smooth everywhere and contains a small spike at the location of the highest discharge. In general, the rising and falling hydrograph limbs in both rainfall scenarios and in all sites are similar and the curves practically overlap each other. The largest differences in magnitude are concentrated around the peak discharge. To better understand how bed evolution affects flow dynamics we integrated the hydrographs in time and obtained the cumulative flow volume (subpanel cumulative flow volume, Figure 10 b). When considering the no bed evolution case, approximately after one day of simulation the flow exiting the watershed (sampled at the outlet) is equal to the volume of water that entered as precipitation (36·104 m3). This proves that mass is well preserved in OverlandFlow and that the square watershed has no sinkholes. However, a difference of 19.4·103 and 48.6·103 m3 between the predicted cumulative flow volume and total rainfall is observed in the uniform and intermittent cases, respectively, when we let the bed evolve. This is not a mass conservation problem but rather a consequence of erosion and deposition patterns within the basin. In the selected sites riverBedDynamics predicts only deposition of sediment, except for Site 3 in the uniform rainfall intensity where the change is practically null. For these places, the intermittent scenario generates consistently larger depositions compared to the uniform case. Most of the bed elevation changes occurred during the first 3 hours of simulation, where the larger discharges and therefore larger shear stresses occurred. In other locations within the watershed we observed scour and deposition of sediments (Figure 11 a and b).

The differences between the predicted cumulative flow volume and total rainfall after 24 hours of simulations can be explained by the morphological changes within the watershed. Here, we analyzed in more detail the intermittent case, but the same analysis is valid for the uniform scenario. When we let the bed evolve different patterns of erosion and deposition were created in the basin (Figure 11 a). In the intermittent case bed surface elevation changes ranged approximately between -2 to 2 m (±1.25 m in the uniform case). This combination of local scour and deposition affected the local flow and created zones that retained large volumes of water (Figure 11 b and c) in quantities practically equal to the volume differences (48.6·103 m3 for the intermittent case). They are not equal because after 24 hours the discharge flowing at the outlet is still 0.052 m3/s and reaches zero at around 36 hours. Most of the nodes where erosion or deposition was predicted are located close to confluences or regions where there are changes in the local channel streamwise direction. The total area that had erosion or deposition larger than 1 cm is 0.2907 km2.

More background on the model used here and the results presented will be published in the future and the citation will be added here:

**TITLE HERE**

The code used in this exercise is taken from the above reference.



**Now on to the code.**
* Below we import Landlab components, functions for importing data, numpy and plotting tools. You should not need to change this.

In [ ]:
## Code Block 1

""" Loads components and other libraries"""
%reset -f
import numpy as np
import pandas as pd
import copyhttp://localhost:8888/edit/MultiGrain/jupyter_example/case4_LargeWatershed/Case4a_noBedEvolution/Case4a_2_intermittent/squareBasinTestDriverFile.ipy
import os
import shutil
from matplotlib import pyplot as plt
from landlab.components import OverlandFlowSpatiallyVariableInputs, RiverBedDynamics
from landlab.io import read_esri_ascii
#from landlab import imshow_grid
from saveRaster import saveRaster

Now we import the data for the watershed we want to route flow on, as well as some model parameters 

In [ ]:
## Code Block 2


""" Numerical simulation conditions and time control settings"""
bedElevation = 'Square_TestBasin.asc'         # ASCII raster DEM containing the bed surface elevation
rainfallFile = 'rainfallIntensity.xlsx'
gsd = pd.read_excel('bedGSD.xlsx',sheet_name='GSD',skiprows=0).values

dtPrecision = 3         # Avoids rounding errors
max_dt = 1           # Overland flow will use the min time step between this value and the automatically calculated. Use seconds.
tPlot = 1800            # Plots will be obtained every this seconds
storeData = 5           # Stores results every this time
tmax = 24*3600+max_dt   # Maximum simulation time, adding max_dt ensures that the last time is stored

# Flow, bed, and upstream simulation conditions 
n = 0.035   # Manning's n


In [ ]:
## Code Block 3

# Remove previous figures
directory = os.getcwd() ; test = os.listdir( directory )

for item in test:
    if item.endswith(".png"):
        os.remove( os.path.join( directory, item ) )
    if item.endswith(".txt"):
        os.remove( os.path.join( directory, item ) )  

In [ ]:
## Code Block 4

# Creates fields and instantiate the component
OverlandFlowSpatiallyVariableInputs.input_var_names
RiverBedDynamics.input_var_names
(rmg, z) = read_esri_ascii(bedElevation, name='topographic__elevation')
rmg.add_zeros('bed_surface__roughness', at = 'link')
rmg.add_zeros('surface_water__depth', at = 'node')
rmg.add_zeros('rainfall__intensity', at = 'node')
rmg['node']['bed_surface__grainSizeDistribution_location'] = np.zeros_like(z)     
   


In [ ]:
## Code Block 6

# Instantiation of components
of = OverlandFlowSpatiallyVariableInputs(rmg, dt_max=max_dt, alpha=0.3,steep_slopes=True)
rbd = RiverBedDynamics(rmg , gsd = gsd, variableCriticalShearStress = True)

In [ ]:
## Code Block 7

# Set boundaries as closed boundaries, the outlet is set to an open boundary. 
#rmg.set_watershed_boundary_condition_outlet_id([69,104], z, 45.)
rmg.set_watershed_boundary_condition(z)

# Create bed and flow initial condition
rmg['link']['bed_surface__roughness'] = np.zeros(rmg.number_of_links) + n   # n is Manning's roughness coefficient
                        # Assigns fixed locations to landlab grid 


In [ ]:
## Code Block 8

rainfallIntensity = pd.read_excel(rainfallFile)
pTime=rainfallIntensity.values[:,0]
pI=rainfallIntensity.values[:,1] * (2.77778 * 10 ** -7)  # rainfall in m/s
p_i = 0                                # current index for time

In [ ]:
## Code Block 9

""" Defines some variables to store data and run the actual simulation """
storeNow = True
plotNow = True                          # Used to save the plot at time zero
check_tmax = True
tPlotOrg=copy.deepcopy(tPlot)           # A copy of tPlot
storeDataOrg=copy.deepcopy(storeData)   # A copy of tPlot
outputFolder = 'output'
cwd = os.getcwd()

if os.path.exists(outputFolder):
    print('The folder') 
    print(outputFolder)
    print('Exists and it will be removed \n');
    shutil.rmtree(outputFolder)     
os.mkdir(outputFolder)

In [ ]:
## Code Block 10

""" Defines some variables to store data and run the actual simulation """
storeNow = True
plotNow = True                          # Used to save the plot at time zero
check_tmax = True
tPlotOrg=copy.deepcopy(tPlot)           # A copy of tPlot
storeDataOrg=copy.deepcopy(storeData)   # A copy of tPlot
linkList = np.arange(69,103)           # This is just to gather data at links in the long profile
nodeList = np.arange(35,70)           # This is just to gather data at data in the long profile
outputFolder = 'output'
cwd = os.getcwd()


In [ ]:
""" Now runs the actual simulation """
t = 0                                   # Initializates the variable
while t < tmax:
    
    rbd.t = t           # Current simulation time
    
    #Calculates the rainfall intensity - variable in time
    if (t >= pTime[p_i+1]):
        rmg['node']['rainfall__intensity'] =  np.zeros(rmg.number_of_nodes) + pI[p_i+1]
        p_i += 1
    else:
        rmg['node']['rainfall__intensity'] = np.zeros(rmg.number_of_nodes) + pI[p_i+1]
    
    of.overland_flow()  # Runs overland flow for one time step
    if t == 0:
        rbd.run_one_step()  # Runs riverBedDynamics for one time step
    
    ## Stores results
    storeData = round(storeData-of.dt, dtPrecision)
    if (storeData <=0) or storeNow:
        os.chdir(outputFolder)
        print('Elapsed time :',np.round(t,1),' s. Current dt =',\
              np.round(of.dt,1),'. Adaptive time =',np.round(of._adaptive_dt,1),' s - Saving data \n')
        data = np.hstack([t,(np.abs(of._q[link_to_sample] * rmg.dx))])
        data = np.reshape(data,[1,data.shape[0]])
        with open("output0_links_surface_water__discharge.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')
        data = np.hstack([t,(of._h[node_to_sample])])
        data = np.reshape(data,[1,data.shape[0]])
        with open("output1_node_surface_water__depth.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')      
        data = np.hstack([t,rmg.at_node["topographic__elevation"][node_to_sample]])
        data = np.reshape(data,[1,data.shape[0]])
        with open("output3_node_topographic__elevation.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')  
        storeData = round(storeDataOrg, dtPrecision)
        storeNow = False
        os.chdir(cwd)

    tPlot = round(tPlot-of.dt, dtPrecision)
    if tPlot <= 0  or plotNow:
        os.chdir(outputFolder)
        
        print('Elapsed time :',np.round(t,1),' s. Current dt =',\
              np.round(of.dt,1),'. Adaptive time =',np.round(of._adaptive_dt,1),' s - Saving plot \n')
        
        # Water depth raster
        filename='depth_'+str(np.round(t,0))+'.asc'    
        data = rmg.at_node["surface_water__depth"]
        saveRaster(data,filename,rmg)
        
        #Bed surface elevation plot
        filename='topographicElevation_'+str(np.round(t,0))+'.asc'    
        data = rmg.at_node["topographic__elevation"]
        saveRaster(data,filename,rmg)
        
        #Bed surface variation plot
        filename = 'topographicVariation_'+str(np.round(t,0))+'.asc'    
        data = rmg.at_node["topographic__elevation"] - rmg.at_node['topographic__elevation_original'] 
        saveRaster(data,filename,rmg)

        x, y0, y1, y2 , y3, y4 = np.loadtxt('output0_links_surface_water__discharge.txt', delimiter=' ', unpack=True)
        plt.plot(x,y0,x,y1,x,y2,x,y3,x,y4)
        plt.xlim([0,18*3600])
        plt.ylim([0,50])
        plt.show()    

        plotNow = False
        tPlot = tPlotOrg
        os.chdir(cwd)

    # Updating t
    if (t + of.dt > tmax) and check_tmax:
        of.dt = tmax - t
        t = tmax
        storeDataNow = True  
        plotNow = True
        check_tmax = False
    else:
        t = round(t + of.dt, dtPrecision)  
    

In [ ]:
###
"""

THIS IS OLD CODE FROM TUTORIAL ONE, I LEFT IT TO CHANGED LATER TO MAKE SOME NEW PLOTS


## Code Block 11

# Results
## Water depth at the end of the simulation
data = np.loadtxt('output1_node_surface_water__depth.txt')
nodesToSample = np.arange(rmg.number_of_node_columns+1,2*rmg.number_of_node_columns)
x = np.arange(0,(rmg.number_of_node_columns-1)*rmg.dx,rmg.dx)

hSample = data[-1,nodesToSample]

hAnalytical = np.zeros_like(x)+h_init
hAnalytical[np.where(x<u*tmax)] =(-7/3 * (n**2 * u**2 * (x[np.where(x<u*tmax)]-u*tmax)))**(3/7)

plt.figure(1)
plt.plot(x, hSample, color='mediumblue')
plt.plot(x, hAnalytical, color='black')
plt.ylabel('Water depth (m)')
plt.xlabel('x (x)')
plt.ylim(0,2.5)
plt.xlim(0,5000)
plt.title('Water depth at 3600 s')
plt.savefig('WaterDepth.png',dpi=300);plt.close()
"""  